## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

from pathlib import Path

import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

import warnings 

In [3]:
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020 

In [5]:
algo_name = 'cnn'
feature_name = 'tfidf-pca' #학습할 피처- tfidf 피처를 pca한 데이터
feature_target = 'feature_target'

model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv' #학습할 피처 가져오기
feature_target_file = feature_dir / f'{feature_target}.csv' #target 데이터 가져오기

p_val_ver7_file = val_dir / f'{model_name}_oof_pred_ver7.csv'
p_tst_ver7_file = tst_dir / f'{model_name}_test_pred_ver7.csv'
sub_ver7_file = sub_dir / f'{model_name}_ver7.csv'

## 데이터 셋 불러오기

In [6]:
X = pd.read_csv(feature_file, index_col=0)
y = pd.read_csv(feature_target_file,index_col=0).values

X_7 = X.iloc[:54879].values
X_tst_7 = X.iloc[54879:].values

In [7]:
print(X_7.shape)
print(X_tst_7.shape)
print(type(X_7))

(54879, 500)
(19617, 500)
<class 'numpy.ndarray'>


In [8]:
X_7

array([[-9.60182778e-02,  1.35410949e-02, -6.60503286e-02, ...,
         1.57118632e-02,  2.44125635e-02, -2.28590565e-02],
       [ 1.74200713e-01, -8.09902055e-02,  6.68472899e-02, ...,
        -5.03131627e-03,  1.66919764e-03,  5.90301730e-05],
       [-1.08924886e-01,  6.13198590e-02, -4.60606378e-02, ...,
        -9.58292290e-03,  6.45213673e-03,  2.15633840e-02],
       ...,
       [-1.19420009e-01, -1.24207603e-01,  2.51135526e-03, ...,
         4.91452690e-03,  5.56308485e-03, -5.83548313e-04],
       [ 2.62578871e-01, -1.01706863e-01,  8.66752065e-02, ...,
        -4.21159256e-04,  3.75348681e-02,  1.00198717e-02],
       [-6.30463041e-02,  2.93850081e-02, -4.86881601e-02, ...,
        -3.69451299e-02,  8.40587639e-03, -5.72133739e-03]])

In [9]:
y

array([[3],
       [2],
       [1],
       ...,
       [1],
       [3],
       [0]])

## cnn 모델 학습

In [10]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [11]:
def get_model(number):
    inputs = Input(batch_shape=(None, number, 1))
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(inputs)
    x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam')
    return model

In [19]:
p_val_ver7 = np.zeros((X_7.shape[0], n_class))
p_tst_ver7 = np.zeros((X_tst_7.shape[0], n_class))


for X, test in [(X_7, X_tst_7)]:  
    for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'Training model for CV #{i_cv}')
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        X = X.reshape(X.shape[0], X.shape[1], 1)
        test = test.reshape(test.shape[0], test.shape[1],1)
        
        clf = get_model(X.shape[1])
        clf.fit(X[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(X[i_val], to_categorical(y[i_val])),
            epochs=100,
            batch_size=64,
            callbacks=[es])
       
        # Predict
        if X.shape[1]==500:
            p_val_ver7[i_val, :] = clf.predict(X[i_val])
            p_tst_ver7 += clf.predict(test) / n_class
        
            
        del clf
        clear_session()
        gc.collect()
            
    print("Training has finished")
    print("*"*100)

            



Training model for CV #1
Epoch 1/100
686/686 [==============================] - 3s 5ms/step - loss: 1.3340 - val_loss: 1.1542
Epoch 2/100
686/686 [==============================] - 3s 5ms/step - loss: 1.1089 - val_loss: 1.1094
Epoch 3/100
686/686 [==============================] - 3s 5ms/step - loss: 1.0408 - val_loss: 1.0602
Epoch 4/100
686/686 [==============================] - 3s 5ms/step - loss: 0.9995 - val_loss: 1.0340
Epoch 5/100
686/686 [==============================] - 3s 5ms/step - loss: 0.9678 - val_loss: 1.0284
Epoch 6/100
686/686 [==============================] - 3s 5ms/step - loss: 0.9414 - val_loss: 1.0063
Epoch 7/100
686/686 [==============================] - 3s 5ms/step - loss: 0.9174 - val_loss: 1.0079
Epoch 8/100
686/686 [==============================] - 3s 5ms/step - loss: 0.9029 - val_loss: 1.0014
Epoch 9/100
686/686 [==============================] - 3s 5ms/step - loss: 0.8792 - val_loss: 1.0009
Epoch 10/100
686/686 [==============================] - 3s 5ms/ste

686/686 [==============================] - 3s 5ms/step - loss: 0.8382 - val_loss: 0.9724
Epoch 11/100
686/686 [==============================] - 3s 5ms/step - loss: 0.8209 - val_loss: 0.9982
Epoch 12/100
686/686 [==============================] - 3s 5ms/step - loss: 0.8073 - val_loss: 0.9789
Epoch 13/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7878 - val_loss: 0.9854
Epoch 14/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7744 - val_loss: 0.9998
Epoch 15/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7620 - val_loss: 1.0022
Epoch 16/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7453 - val_loss: 1.0131
Epoch 17/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7317 - val_loss: 1.0250
Epoch 18/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7174 - val_loss: 1.0116
Epoch 19/100
686/686 [==============================] - 3s 5ms/step - loss: 0.7043 - val_loss: 

In [18]:
print(f'lr ver7 Accuracy (CV): {accuracy_score(y, np.argmax(p_val_ver7, axis=1)) * 100:8.4f}%')
print(f'lr ver7 Log Loss (CV): {log_loss(y, p_val_ver7):8.4f}')

lr ver7 Accuracy (CV):  62.3554%
lr ver7 Log Loss (CV):   0.9798


## 제출 파일 생성 및 기타 파일 생성

In [14]:
# submission 파일 생성

sub = pd.read_csv(sample_file, index_col=0)

# Ver7
sub[sub.columns] = p_tst_ver7
sub.to_csv(sub_ver7_file)



In [15]:
# p_val 파일 생성 -> oof

# Ver7
np.savetxt(p_val_ver7_file, p_val_ver7, fmt='%.18f', delimiter=',')



In [16]:
# p_tst 파일 생성 -> test 

# Ver7
np.savetxt(p_tst_ver7_file, p_tst_ver7, fmt='%.18f', delimiter=',')
